# Bankrupt dataset ETL focused on financial raw values

Librerie varie da installare

In [ ]:
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn

Inclusione delle librerie utilizzate

In [ ]:
import pandas as pd
import os
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
# Change plot output format
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import seaborn as sns

Variabili di gestione files

In [ ]:
# Path of the directory containing the bankruptcy companies .csv files, can be changed
PATH_BANKRUPT = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Bankruptcy_rielaborati_G"

# True = export summary file in the OUTPUT_PATH
to_export = False

# Path of the desired output file, can be changed
OUTPUT_PATH = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset_output"

Leggo i file .csv contenuti in una directory e le sue sottodirectories (BANKRUPT)

In [ ]:
# Dataset with all the csv files
input_dataset = pd.DataFrame()

# Path of each csv file
# Path description: Bankruptcy_rielaborate_G/*.csv
csv_files = glob.glob(os.path.join(PATH_BANKRUPT, "**/*.csv"), recursive=True)

# Loop over the list of csv files
for f in csv_files:

    # Print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])

    # Read each csv file
    partial_dataset = pd.read_csv(f, sep=";", dtype=object, thousands=",", decimal=".")

    # Append the partial dataset
    input_dataset = pd.concat([input_dataset, partial_dataset])




print("Dimensione dataset delle società in bancarotta: ", input_dataset.shape)

In [ ]:
input_dataset.head()

Gestisco i nomi delle colonne in modo consistente

In [ ]:
for col in input_dataset.columns:
    old_col_name = col
    new_col_name = re.sub("\r\nEUR\r\nYear - \d|\r\nYear - \d|\nEUR\nYear - \d|Year - \d|EUR", "", old_col_name)
    new_col_name = re.sub("\r\n|\r|\n", " ", new_col_name)
    input_dataset.rename(columns={old_col_name: new_col_name}, inplace=True)

In [ ]:
# Combine all columns with the same name
input_dataset = input_dataset.groupby(level=0, axis=1).sum()

Seleziono le colonne contenti le varie voci di bilancio

In [ ]:
# Massive amount of columns needed
to_keep_columns = ['Company name',
                   'Province',
                   'Employees',
                   'Accounting closing date Last avail. yr',
                   'Legal form',
                   'Legal status',
                   'Total financial charges',
                   "TOTAL SHAREHOLDERS' FUNDS",
                   'TOTAL CURRENT ASSETS',
                   'OPERATING MARGIN',
                   "Total depreciation, amortization and writedowns",
                   'Provisions fo risks and charges',
                   'Other provisions',
                   'Tax payable',
                   'Tax payable beyond 12 months',
                   'Due to social security institutions',
                   'Due to social security institutions - beyond 12 months',
                   'TOTAL FIXED ASSETS',
                   'PROFIT (LOSS) GROUP',
                   'Trade accounts',
                   'Trade accounts - beyond 12 months',
                   'Due to suppliers',
                   'Due to suppliers - beyond 12 months',
                   'TOTAL INVENTORIES',
                   'Due to banks',
                   'Due to banks - beyond 12 months',
                   'Due to other lenders',
                   'Due to other lenders - beyond 12 months',
                   'Bonds',
                   'Bonds beyond 12 months',
                   'Due to shar. for loans',
                   'Due to sharesholders for loans - beyond 12 months',
                   'Revenues from sales and services',
                   'TOTAL LIQUID FUNDS',
                   'TOTAL PAYABLES',
                   'Total payables during period',
                   'Total payables after period',
                   'TOTAL ASSETS',
                   "TOTAL LIABILITIES AND SHAREHOLDERS' FUNDS",
                   'Raw, consum. mat. and goods for resale',
                   'Services']

input_dataset = input_dataset[to_keep_columns]

In [ ]:
input_dataset.head()

Rinomino certe colonne per essere consistente con il dataset delle companie in attivo

In [ ]:
map_columns = {
    "Company name": "Ragione sociale",
    "Employees": "Number of employees",
    "Legal form": "Legal Form",
    "Legal status": "Legal Status",
    "TOTAL SHAREHOLDERS' FUNDS": "TOTAL SHAREHOLDERS FUNDS",
    "Accounting closing date Last avail. yr": "Accounting closing date",
    "TOTAL ASSETS": "Total asset",
    "TOTAL LIABILITIES AND SHAREHOLDERS' FUNDS": "TOTAL LIABILITIES AND SHAREHOLDERS FUNDS",
    "Due to shar. for loans": "Due to shareholders for loans"
}
input_dataset.rename(columns=map_columns, inplace=True)

In [ ]:
input_dataset.head()

Tutti i valori non disponibili sono rimpiazzati con NaN

In [ ]:
input_dataset.replace(to_replace="n.a.", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="n.d.", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="#VALUE!", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="#NOME?", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="UDATACHI", value=np.NaN, inplace=True)
input_dataset.isna().sum()

Rimuovo tutti i record contenenti valori NaN

In [ ]:
input_dataset.dropna(inplace=True)
input_dataset.isna().sum()

In [ ]:
print("Dimensione dataset filtrato dai valori mancanti e con numero di colonne ridotte: ", input_dataset.shape)

Cast delle colonne del dataset a tipi numerici e descrittivi (stringhe)

In [ ]:
str_cols = ['Ragione sociale',
            'Province',
            'Legal Form',
            'Legal Status',
            'Accounting closing date']

for col in input_dataset.columns:
    if col in str_cols:
        input_dataset[col] = input_dataset[col].astype(str)
    else:
        input_dataset[col] = pd.to_numeric(input_dataset[col])

Rendo la colonna Legal Status contenente un unico valore (Bankruptcy)

In [ ]:
input_dataset["Legal Status"] = "Bankruptcy"

Sostituisco i valori del campo “Legal Form” con “Società di capitali”, “Società di persone” o “Altro”

In [ ]:
# Map società di capitali
societa_capitali_values = ["S.R.L.",
                           "Limited liability company - SRL",
                           "One-person company with limited liability - SRL",
                           "S.R.L. a socio unico",
                           "S.R.L. semplificata",
                           "S.C.A.R.L.",
                           "Cooperative company with limited liability – SCARL",
                           "Limited liability company, simplified",
                           "Società consortile a responsabilita' limitata",
                           "S.P.A.",
                           "S.C.A.R.L.P.A.",
                           "Joint stock company - SPA",
                           "Cooperative company with limited liability by shares - SCARLPA",
                           "S.P.A. a socio unico",
                           "One-person joint stock company - SPA",
                           "Società consortile per azioni",
                           "Consortium by shares",
                           "PICC. S.C.A.R.L.",
                           "Cooperative company with limited liability, small - SCARL",
                           "Limited liability consortium cooperative company",
                           "Società consortile cooperativa a responsabilità limitata",
                           "S.A.P.A.",
                           "S.R.L. a capitale ridotto",
                           "Public limited company - SA",
                           "Limited partnership with shares - SAPA",
                           "S.C.A.R.I.",
                           "Consortium",
                           "Consorzio",
                           "Società di capitali"]

# Dictionary società di capitali
dict_capitali = dict.fromkeys(societa_capitali_values, "Società di capitali")

# Map società di persone
societa_persone_values = ["Limited partnership - SAS",
                          "S.N.C.",
                          "Società semplice",
                          "General partnership - SNC",
                          "Partnership - SS",
                          "S.A.S.",
                          "Cooperativa sociale",
                          "Social cooperative company",
                          "Società cooperativa consortile",
                          "Consortium of cooperatives",
                          "Small cooperative company",
                          "Società di persone"]

# Dictionary società di persone
dict_persone = dict.fromkeys(societa_persone_values, "Società di persone")

# General dictionary
dict = {**dict_capitali, **dict_persone}

# Apply the dictionary and apply the value "Altro" when it is not a dictionary's value
input_dataset["Legal Form"] = input_dataset["Legal Form"].map(dict).fillna("Altro").astype(str)

Aggiungo colonna denominata “Company Size” per distinguere tra “Grandi”, “Medie” e “Piccole” imprese

In [ ]:
# Define company size label based on some column values
def label_size(row):
    score_grandi = 0
    score_medie = 0
    score_piccole = 0

    if row["Revenues from sales and services"] >= 50000000:
        score_grandi += 1
    elif 10000000 <= row["Revenues from sales and services"] < 50000000:
        score_medie += 1
    else:
        score_piccole += 1

    if row["Total asset"] >= 43000000:
        score_grandi += 1
    elif 10000000 <= row["Total asset"] < 43000000:
        score_medie += 1
    else:
        score_piccole += 1

    if row["Number of employees"] >= 250:
        score_grandi += 1
    elif 50 <= row["Number of employees"] < 250:
        score_medie += 1
    else:
        score_piccole += 1

    score_max = max(score_grandi, score_medie, score_piccole)

    if score_grandi == score_max:
        return "Grandi"
    elif score_medie == score_max:
        return "Medie"
    else:
        return "Piccole"

In [ ]:
input_dataset["Company Size"] = input_dataset.apply(lambda row: label_size(row), axis=1)

Inizio a guardare come sono correlate le varie voci di bilancio

In [ ]:
corr = input_dataset.corr()

sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
plt.subplots(figsize=(30, 30))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1,
            center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Inizio a rimuovere le voci con correlazione elevata (> |0.75|)

In [ ]:
removed_columns =  [# Removed from active companies correlation analysis
                    "Total asset",
                    "TOTAL SHAREHOLDERS FUNDS",
                    "TOTAL PAYABLES",
                    "TOTAL INVENTORIES",
                    "TOTAL LIABILITIES AND SHAREHOLDERS FUNDS",
                    "Revenues from sales and services",
                    "Raw, consum. mat. and goods for resale",
                    "Total payables during period",
                    # Removed from bankrupt companies correlation analysis
                    "Total payables after period",
                    "Trade accounts"
]

filtered_dataset = input_dataset.drop(removed_columns, axis=1)

In [ ]:
filtered_corr = filtered_dataset.corr()

sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(filtered_corr, dtype=bool))

# Set up the matplotlib figure
plt.subplots(figsize=(30, 30))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(filtered_corr, mask=mask, cmap=cmap, vmax=1,
            center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Controllo il dataset con certe voci di bilancio altamente correlate rimosse

In [ ]:
filtered_dataset.head()

In [ ]:
filtered_dataset.describe().T

Esporto in csv e pickle

In [ ]:
if to_export:
    filtered_dataset.to_csv(OUTPUT_PATH + "/bankruptcy_raw.csv")
    filtered_dataset.to_pickle(OUTPUT_PATH + "/bankruptcy_raw.pkl")